In [19]:
import cv2
import mediapipe as mp
import time
import numpy as np
from datetime import datetime
import os

# Initialize MediaPipe Gesture Recognizer with proper imports
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Global variables for timer and gesture tracking
class TimerState:
    def __init__(self):
        self.timer_running = False
        self.start_time = 0
        self.elapsed_time = 0
        self.timer_stopped = False
        self.previous_gesture = None
        self.last_gesture_time = 0
        self.gesture_duration_threshold = 0.2

timer_state = TimerState()
latest_result = None  # Global variable to store the latest result

def draw_landmarks(image, hand_landmarks):
    """
    Draw hand landmarks on the image.
    """
    for landmark in hand_landmarks:
        x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)

def calculate_finger_angle(a, b, c):
    """
    Menghitung sudut yang dibentuk oleh tiga titik (dalam derajat)
    a: titik pertama
    b: titik tengah (vertex)
    c: titik ketiga
    """
    ba = np.array([a.x - b.x, a.y - b.y])
    bc = np.array([c.x - b.x, c.y - b.y])
    
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine, -1.0, 1.0))
    
    return np.degrees(angle)

def is_finger_extended(mcp, pip, tip, threshold=175.0):
    """
    Mengecek apakah sebuah jari terbuka berdasarkan sudut yang dibentuk
    mcp: Meta-Carpophalangeal joint (pangkal jari)
    pip: Proximal Interphalangeal joint (sendi jari tengah)
    tip: ujung jari
    """
    angle = calculate_finger_angle(mcp, pip, tip)
    return angle > threshold

def is_hand_extended(landmarks):
    """
    Mengecek apakah tangan dalam posisi terbuka (open palm)
    dengan memeriksa posisi setiap jari
    """
    # Cek jari telunjuk
    index_extended = is_finger_extended(
        landmarks[5],  # INDEX_FINGER_MCP
        landmarks[6],  # INDEX_FINGER_PIP
        landmarks[8]   # INDEX_FINGER_TIP
    )
    
    # Cek jari tengah
    middle_extended = is_finger_extended(
        landmarks[9],   # MIDDLE_FINGER_MCP
        landmarks[10],  # MIDDLE_FINGER_PIP
        landmarks[12]   # MIDDLE_FINGER_TIP
    )
    
    # Cek jari manis
    ring_extended = is_finger_extended(
        landmarks[13],  # RING_FINGER_MCP
        landmarks[14],  # RING_FINGER_PIP
        landmarks[16]   # RING_FINGER_TIP
    )
    
    # Cek jari kelingking
    pinky_extended = is_finger_extended(
        landmarks[17],  # PINKY_MCP
        landmarks[18],  # PINKY_PIP
        landmarks[20]   # PINKY_TIP
    )
    
    # Tangan dianggap terbuka jika minimal 3 jari terbuka
    extended_fingers = sum([index_extended, middle_extended, ring_extended, pinky_extended])
    return extended_fingers >= 4

def process_result(result, output_image, timestamp_ms):
    global timer_state, latest_result
    
    latest_result = result
    current_time = time.time()

    if hasattr(result, 'gestures') and result.gestures:
        # Get the most likely gesture
        gesture = result.gestures[0][0].category_name
        confidence = result.gestures[0][0].score
    
    if hasattr(result, 'hand_landmarks') and result.hand_landmarks:
        # Gunakan fungsi is_hand_extended untuk mendeteksi posisi tangan
        hand_extended = is_hand_extended(result.hand_landmarks[0])
        
        if hand_extended:
            if not timer_state.timer_running and not timer_state.timer_stopped:
                    timer_state.timer_running = True
                    timer_state.start_time = current_time
                    print("Timer Started!")
                    timer_state.previous_gesture = "Open Palm"

        elif gesture == "Closed_Fist":
            if timer_state.timer_running:
                    timer_state.timer_running = False
                    timer_state.timer_stopped = True
                    timer_state.elapsed_time = current_time - timer_state.start_time
                    timer_state.previous_gesture = "Closed_Fist"
                    print(f"Timer Stopped! Elapsed Time: {timer_state.elapsed_time:.2f}s (Confidence: {confidence:.2f})")
        
        timer_state.last_gesture_time = current_time

def calculate_velocity(time, distance):
    """
    Menghitung estimasi kecepatan anak panah
    Parameter:
    1. time: Waktu tempuh anak panah. Dalam source code ini, variabel yang menyimpan waktu tempuh ialah: timer_state.elapsed_time
    2. distance: Jarak antara pemanah dengan target. Isi dengan hasil inputan dari User.
    """
    return distance/time


def main():
    global timer_state, latest_result
    
    # Set up camera
    cap = cv2.VideoCapture(0)
    
    # Get the current script's directory
    script_dir = os.getcwd()
    model_path = os.path.join(script_dir, 'gesture_recognizer.task')

    if not os.path.exists(model_path):
        print(f"Error: Model file not found at {model_path}")
        print("Please download the model using:")
        print("wget -O gesture_recognizer.task https://storage.googleapis.com/mediapipe-models/gesture_recognizer/gesture_recognizer/float16/1/gesture_recognizer.task")
        return
    
    # Initialize the Gesture Recognizer
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.GestureRecognizerOptions(
        base_options=base_options,
        running_mode=vision.RunningMode.LIVE_STREAM,
        result_callback=process_result,
        min_hand_detection_confidence=0.5,
        min_hand_presence_confidence=0.5,
        min_tracking_confidence=0.5
    )
    
    try:
        with vision.GestureRecognizer.create_from_options(options) as recognizer:
            print("Gesture Recognizer initialized successfully")
            
            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    print("Failed to capture frame")
                    continue
                
                # Convert the BGR image to RGB
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
                
                # Process the frame
                recognizer.recognize_async(mp_image, int(time.time() * 1000))
                
                # Draw landmarks if hand landmarks are available
                if latest_result and hasattr(latest_result, 'hand_landmarks') and latest_result.hand_landmarks:
                    draw_landmarks(image, latest_result.hand_landmarks[0])
                
                # Timer display
                timer_position = (image.shape[1] - 150, image.shape[0] - 30)
                
                if timer_state.timer_running:
                    current_time = time.time()
                    timer_state.elapsed_time = current_time - timer_state.start_time
                
                # Display timer
                timer_text = f"Time: {timer_state.elapsed_time:.2f}s"
                cv2.putText(image, timer_text, timer_position,
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                
                # Display status
                if timer_state.timer_stopped:
                    status_text = "Recording Complete - Session Ended"
                elif timer_state.timer_running:
                    status_text = "Timer Running - Make closed fist to stop"
                else:
                    status_text = "Ready - Show open palm to start timer"
                
                cv2.putText(image, status_text, (10, 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Display current gesture
                if timer_state.previous_gesture:
                    gesture_text = f"Gesture: {timer_state.previous_gesture}"
                    cv2.putText(image, gesture_text, (10, 90),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                cv2.imshow('Gesture Timer', image)
                if cv2.waitKey(5) & 0xFF == 27:  # Press ESC to exit
                    break
                
    except Exception as e:
        print(f"Error during execution: {str(e)}")
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Gesture Recognizer initialized successfully
Timer Started!
Timer Stopped! Elapsed Time: 2.35s (Confidence: 0.89)
